# 单元课题：XYG3 能量计算 (1)

在结束这一单元前，我们通过完成一个比较完整的项目，回顾公式记号与最为基础的程序实现。这个比较完整的项目就是计算 XYG3 能量。

这个课题要求，除了 **SCF 初猜密度、电子积分、轨道与泛函格点** 外，尽可能只使用 **不超过** numpy 的工具。

我们以后可能会使用一些程序上的技巧、以及 pyxdh 中所提供的一些便利工具来缩短文档和代码的篇幅；但作者认为，若要成为程序开发者，需要对一些必要的底层方法进行了解。这是作者编写这份课题的初衷。

作者认为，这份课题的所有代码未必需要亲手写一遍；这篇文档的代码前都会有导引，若看到导引就能知道代码大致是怎么写的 (调用哪个函数、或者能查阅到以前阅读过的哪篇文档的哪一小节、或者能正确地查阅到 PySCF 的 API 文档)，我认为就达成了作者期望读者阅读文档的目的了。

## 程序流程导引

在设计程序之前，我们要知道 XYG3 的能量是如何给出的。

- 首先，我们需要跑一次 B3LYP 得到其密度矩阵 $D_{\mu \nu}$ 与轨道系数 $C_{\mu p}$；

- 其次，我们将密度矩阵代入 XYG3 的 GGA 分项进行计算，得到其能量的 GGA 部分；

- 最后，将 B3LYP 得到的轨道系数代入 PT2 计算，得到 XYG3 能量的 PT2 部分。

程序分为以下几个模块：

1. 初始化

    - 引入库 (PySCF)
    
    - 定义分子 (PySCF)
    
    - 定义格点 (PySCF)

2. 无需自洽场密度或轨道系数就能计算的变量 (自洽场无关)
    
    - 原子轨道积分 (PySCF)
    
    - 轨道格点与格点权重 (PySCF)
    
    - 原子核排斥能 $E_\mathrm{nuc}$
    
3. 需要代入自洽场密度或轨道系数的变量 (自洽场相关)

    - 库伦、交换积分
    
    - 密度格点与泛函格点 (PySCF)
    
    - DFT 势矩阵
    
    - GGA 能量与 SCF 循环
    
    - PT2 能量
    
    - 最终 XYG3 能量

我们假定内存空间总是足够的。上述标记 PySCF 的部分是指我们允许在这些代码中使用 PySCF 程序，其它部分一概不允许 (包括不允许使用 pyxdh)。依据这些提示，读者应当能大致构思出程序框架，并能在 3 天时间以内从头写一个 XYG3 能量计算程序。

我们下面给出参考程序。

## 初始化部分

### 引入库

在引入 Python 库时，我们要考虑到以下方面：

- 我们需要使用到 PySCF 中的分子定义 `gto`、DFT 计算 `dft` 与 PT2 计算 `mp` 部分

- `np.einsum` 的优化选项 `optimize` 需要常开

- numpy 的输出稍简洁一些，这里使用 5 位小数输出

- 为了减少输出，因此不输出 Python 的 warning 信息

In [1]:
import numpy as np
import warnings
from pyscf import gto, dft, mp
from functools import partial

np.einsum = partial(np.einsum, optimize=["greedy", 1024 ** 3 * 2 / 8])
warnings.filterwarnings("ignore")
np.set_printoptions(5, linewidth=150, suppress=True)

### 定义分子

如以前一样，我们定义如下的双氧水分子，基组为 6-31G：

In [2]:
mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.0  0.0  0.0
H  0.0  0.7  1.0
"""
mol.basis = "6-31G"
mol.verbose = 0
mol.build()

我们以后可能会非常经常地使用占据、非占轨道数量与分割 (slice)，以及原子数量：

- `natm` 原子数

- `nao` 原子轨道数，`nmo` 分子轨道数，一般来说在量化程序中，两者相等

- `nocc` 占据轨道数，`nvir` 非占轨道数

- `so`, `sv`, `sa` 分别代表占据、非占、全轨道的分割

In [10]:
natm = mol.natm
nao = nmo = mol.nao
nocc = mol.nelec[0]
nvir = nmo - nocc
so, sv, sa = slice(0, nocc), slice(nocc, nmo), slice(0, nmo)

### 定义格点

我们定义下述 (99, 590) 格点：

In [3]:
grids = dft.Grids(mol)
grids.atom_grid = (99, 590)
grids.becke_scheme = dft.gen_grid.stratmann
grids.build()

我们也会经常使用格点数量 `ngrids`：

In [13]:
ngrid = grids.weights.size

## 自洽场无关部分

### 原子轨道积分

我们定义下述经常使用的原子轨道积分：

- `T` 动能积分 $t_{\mu \nu} = \langle \mu | - \frac{1}{2} \nabla^2 | \nu \rangle$

- `Vnuc` 外势能积分 $v^\mathrm{nuc}_{\mu \nu} = \langle \mu | - \frac{Z_M}{| \boldsymbol{r} |} | \nu \rangle_{\boldsymbol{r} \rightarrow \boldsymbol{M}}$

- `H_0_ao` Hamiltonian Core 积分 $h_{\mu \nu} = t_{\mu \nu} + v^\mathrm{nuc}_{\mu \nu}$

- `S_0_ao` 重叠积分 $S_{\mu \nu} = \langle \mu | \nu \rangle$

- `eri0_ao` 双电子排斥积分 (ERI) $(\mu \nu | \kappa \lambda)$

我们同时定义 SCF 循环过程中需要使用到的导出量

- `X` $X_{\mu \nu}$，满足 $X_{\mu \kappa} S_{\mu \nu} X_{\nu \lambda} = \delta_{\kappa \lambda}$ 或 $\mathbf{X}^\dagger \mathbf{S} \mathbf{X} = \mathbf{1}$

In [7]:
T = mol.intor("int1e_kin")
Vnuc = mol.intor("int1e_nuc")
H_0_ao = T + Vnuc
S_0_ao = mol.intor("int1e_ovlp")
eri0_ao = mol.intor("int2e")
X_cd = np.linalg.inv(np.linalg.cholesky(S_0_ao).T)

### 轨道格点与权重

格点积分过程中会经常使用 PySCF 的 `NumInt`，在此我们用 `ni` 来表示 `NumInt` 的一个实例：

In [15]:
ni = dft.numint.NumInt()

我们定义权重格点 `weight` $w$；它仅用于与泛函格点乘积，在公式中不会出现：

In [16]:
weight = grids.weights

在计算 DFT 能量的过程中，我们至多使用轨道对电子坐标的一阶梯度。我们将会生成下述轨道格点：

- `ao_0` $\phi_{\mu}$

- `ao_1` $\phi_{r \mu}$

- `ao_01` 是 `ao_0` 与 `ao_1` 的拼接，为密度格点生成所用

In [21]:
ao = np.zeros((4, ngrid, nao))  # 4 refers to (noderiv, x_deriv, y_deriv, z_deriv)
g_start = 0
for inner_ao, _, _, _ in ni.block_loop(mol, grids, nao, deriv=1, max_memory=2000):
    ao[:, g_start:g_start+inner_ao.shape[-2]] = inner_ao
    g_start += inner_ao.shape[-2]
ao_0 = ao[0]
ao_1 = ao[1:4]
ao_01 = np.hstack(([ao_0], ao_1), axis=0)

TypeError: hstack() got an unexpected keyword argument 'axis'

In [20]:
np.array([ao_0]).shape

(1, 130776, 22)